In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

/Users/user/.pyenv/versions/3.9.2/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/user/.pyenv/versions/3.9.2/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <760F3975-CB00-30FA-8785-26E85127AF0E> /Users/user/.pyenv/versions/3.9.2/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <1BD88466-7CFF-3C8A-ADD6-913C4DC2F6C3> /Users/user/.pyenv/versions/3.9.2/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/user/.pyenv/versions/3.9.2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See ht

In [3]:
training_data= datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)
test_data=datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████████████████████| 26421880/26421880 [00:22<00:00, 1195489.26it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████████████████████████████| 29515/29515 [00:00<00:00, 98847.62it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|█████████████████████████████| 4422102/4422102 [00:05<00:00, 884407.50it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████████████████████████████| 5148/5148 [00:00<00:00, 3179543.07it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
batch_size=64

train_dataloader= DataLoader(training_data, batch_size=batch_size)
test_dataloader= DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f'X[N,C,H,W]: {X.shape}')
    print(f'y: {y.shape} {y.dtype}')
    break

X[N,C,H,W]: torch.Size([64, 1, 28, 28])
y: torch.Size([64]) torch.int64


In [7]:
device=("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using {device} device")

Using mps device


In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten= nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x= self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model= NeuralNetwork().to(device)

In [9]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
loss_fn= nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size= len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        X, y= X.to(device), y.to(device)
        pred= model(X)
        loss= loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")        

In [12]:
def test(dataloader, model, loss_fn):
    size=len(dataloader.dataset)
    num_batches= len(dataloader)
    model.eval()
    test_loss, correct= 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X,y= X.to(device), y.to(device)
            pred= model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.310879  [   64/60000]
loss: 2.294036  [ 6464/60000]
loss: 2.275162  [12864/60000]
loss: 2.262870  [19264/60000]
loss: 2.257936  [25664/60000]
loss: 2.223459  [32064/60000]
loss: 2.233027  [38464/60000]
loss: 2.195033  [44864/60000]
loss: 2.183781  [51264/60000]
loss: 2.166316  [57664/60000]
Test Error: 
 Accuracy: 34.9%, Avg loss: 2.154565 

Epoch 2
-------------------------------
loss: 2.163310  [   64/60000]
loss: 2.155412  [ 6464/60000]
loss: 2.099253  [12864/60000]
loss: 2.114857  [19264/60000]
loss: 2.073660  [25664/60000]
loss: 2.006443  [32064/60000]
loss: 2.036215  [38464/60000]
loss: 1.949736  [44864/60000]
loss: 1.948273  [51264/60000]
loss: 1.897035  [57664/60000]
Test Error: 
 Accuracy: 56.7%, Avg loss: 1.887336 

Epoch 3
-------------------------------
loss: 1.915283  [   64/60000]
loss: 1.888590  [ 6464/60000]
loss: 1.773937  [12864/60000]
loss: 1.816654  [19264/60000]
loss: 1.712790  [25664/60000]
loss: 1.661222  [32064/600

In [14]:
torch.save(model.state_dict(), 'model.pth')

In [15]:
model= NeuralNetwork().to(device)
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [17]:
classes=[
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",    
]
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
